This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [4]:
# get api key from your .env file
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path(r"C:\Users\becke\OneDrive\Documents\Jupyter Notebooks\Notes\Basics\Requests\key.env")
load_dotenv(dotenv_path=dotenv_path)
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [11]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [13]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?api_key={API_KEY}"
r = requests.get(url)
json_data = r.json()

In [14]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# for k, v in json_data.items():
    # print(k, v)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [16]:
# Task 1
# Collecting data from FSE for ticker 'AFX_X' for 2017
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}"
r = requests.get(url)
json_data = r.json()

In [17]:
# Task 2
# Converting JSON object into dictionary using definition
def headers(json):
    for k, v in json.items():
        return v


In [18]:
# Task 3
# Finding index for column of interest in our data
json = headers(json_data)
[*enumerate(json['column_names'])]

[(0, 'Date'),
 (1, 'Open'),
 (2, 'High'),
 (3, 'Low'),
 (4, 'Close'),
 (5, 'Change'),
 (6, 'Traded Volume'),
 (7, 'Turnover'),
 (8, 'Last Price of the Day'),
 (9, 'Daily Traded Units'),
 (10, 'Daily Turnover')]

In [19]:
# Setting index value and creating definition to extract prices
open = 1
json = headers(json_data)

def open_prices(json):
    prices = []
    for data in json['data']:
        price = data[open]
        if type(price) == float:
            prices.append(price)
    return prices

open_prices = [price for price in open_prices(json)]

In [20]:
# Highest and lowest opening prices
max_price = max(open_prices)
min_price = min(open_prices)
print(f"The highest price of the stock in this period was: {max_price}")
print(f"The lowest price of the stock in this period was: {min_price}")

The highest price of the stock in this period was: 53.11
The lowest price of the stock in this period was: 34.0


In [21]:
# Task 4
# Setting index values
# Creating defintion to extract high and low prices while also finding their difference
high = 2
low = 3
json = headers(json_data)

def change_high_low(json):
    changes = []
    for data in json['data']:
        high_price = data[high]
        low_price = data[low]
        change = high_price - low_price
        changes.append(change)
    return changes

change_high_low = [change for change in change_high_low(json)]

In [22]:
# Largest change in one day
max_change = max(change_high_low)
print(f"The largest change in one day (based on High and Low price) was: {max_change}")

The largest change in one day (based on High and Low price) was: 2.8100000000000023


In [23]:
# Task 5
# Setting index value
# Creating definitions to extract prices and calculate changes
close = 4
json = headers(json_data)

def closing_prices(json):
    prices = []
    for data in json['data']:
        price = data[close]
        prices.append(price)
    return prices

closing_prices = [price for price in closing_prices(json)]
index1 = [*range(len(closing_prices)-1)]
index2 = [*range(1, len(closing_prices))]

def price_deltas(prices):
    deltas = []
    for i1, i2 in zip(index1, index2):
        delta = abs(prices[i1] - prices[i2])
        deltas.append(delta)
    return deltas

price_deltas = price_deltas(closing_prices)

In [24]:
# Largest change between any two days
max_delta = max(price_deltas)
print(f"The largest change between any two days (based on Closing price) was: {max_delta}")

The largest change between any two days (based on Closing price) was: 2.559999999999995


In [25]:
# Task 6
# Setting index value
# Creating definition to extract volumes and calculate mean
traded_volume = 6
json = headers(json_data)

def trading_volume(json):
    volumes = []
    for data in json['data']:
        volume = data[traded_volume]
        volumes.append(volume)
    return volumes

# What was the average daily trading volume during this year?
trading_volume = trading_volume(json)
print(f"The average daily trading volume during this year was: {sum(trading_volume) / len(trading_volume)}")

The average daily trading volume during this year was: 89124.33725490196


In [26]:
# Task 7
trading_volume = sorted(trading_volume)
median = trading_volume[len(trading_volume)//2]

In [27]:
# Median trading volume
print(f"The median trading volume during this year was: {median}")

The median trading volume during this year was: 76286.0
